In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline



In [22]:
X_train = pd.read_parquet('../data/processed/X_train.parquet')
X_test = pd.read_parquet('../data/processed/X_test.parquet')
y_train = pd.read_parquet('../data/processed/y_train.parquet')
y_test = pd.read_parquet('../data/processed/y_test.parquet')

In [23]:
X_train.drop(columns="code_snippet", inplace=True)
X_test.drop(columns="code_snippet", inplace=True)

In [24]:
X_train.describe()

,loc,num_args,num_returns,num_variables,num_function_calls,estimated_difficulty,estimated_bugs,docstring_length,num_comments,name_length,bad_variable_names_count,max_return_length,comment_density,max_nesting_depth,num_try_blocks,estimated_complexity
count,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000,649538.000000
mean,18.205038,2.019047,0.677588,3.307860,6.312721,0.798108,0.012160,1.763364,0.854230,17.575506,0.569522,21.754578,0.031409,1.679046,0.067417,3.052637
std,32.478218,2.305480,1.094247,6.527018,12.877280,1.484346,0.273198,7.629977,2.993329,12.133326,2.055308,159.823869,0.075358,0.957722,0.338455,4.978079
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
25%,4.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
50%,10.000000,1.000000,0.000000,2.000000,3.000000,0.500000,0.000667,0.000000,0.000000,15.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000
75%,21.000000,2.000000,1.000000,4.000000,7.000000,1.000000,0.005170,1.000000,0.000000,23.000000,0.000000,22.000000,0.000000,2.000000,0.000000,3.000000
max,6072.000000,374.000000,116.000000,817.000000,2714.000000,100.000000,187.266816,838.000000,241.000000,219.000000,339.000000,57944.000000,0.933333,13.000000,25.000000,440.000000


In [25]:
X_train = X_train[~X_train.duplicated()]
X_test = X_test[~X_test.duplicated()]

train_keep_indices = X_train.index
test_keep_indices = X_test.index

# Filter y_train and y_test to match the deduplicated X's
y_train = y_train.loc[train_keep_indices]
y_test = y_test.loc[test_keep_indices]

In [26]:
X_train.describe()

,loc,num_args,num_returns,num_variables,num_function_calls,estimated_difficulty,estimated_bugs,docstring_length,num_comments,name_length,bad_variable_names_count,max_return_length,comment_density,max_nesting_depth,num_try_blocks,estimated_complexity
count,518207.000000,518207.000000,518207.000000,518207.000000,518207.000000,518207.000000,518207.000000,518207.000000,518207.000000,518207.000000,518207.000000,518207.000000,518207.000000,518207.000000,518207.000000,518207.00000
mean,21.361898,2.118746,0.749316,3.874531,7.408763,0.960117,0.014907,2.109518,1.039880,18.655809,0.688966,24.586957,0.036927,1.811014,0.082517,3.48173
std,35.348190,2.464547,1.181844,7.093600,14.118888,1.596794,0.305756,8.422522,3.298904,12.490121,2.265113,177.484873,0.079810,1.003935,0.374062,5.44773
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.00000
25%,6.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.00000
50%,13.000000,2.000000,1.000000,2.000000,4.000000,0.500000,0.001585,0.000000,0.000000,16.000000,0.000000,0.000000,0.000000,2.000000,0.000000,2.00000
75%,25.000000,3.000000,1.000000,5.000000,9.000000,1.142857,0.008422,1.000000,1.000000,24.000000,1.000000,25.000000,0.037037,2.000000,0.000000,4.00000
max,6072.000000,374.000000,116.000000,817.000000,2714.000000,100.000000,187.266816,838.000000,241.000000,219.000000,339.000000,57944.000000,0.933333,13.000000,25.000000,440.00000


In [27]:
y_train.describe()

,quality
count,518207
unique,2
top,bad
freq,266580


In [28]:
X_train.dtypes

loc                           int64
num_args                      int64
num_returns                   int64
num_variables                 int64
num_function_calls            int64
has_decorators                 bool
uses_globals                   bool
is_recursive                   bool
estimated_difficulty        float64
estimated_bugs              float64
has_docstring                  bool
docstring_length              int64
num_comments                  int64
name_length                   int64
is_name_well_formed            bool
bad_variable_names_count      int64
max_return_length             int64
comment_density             float64
max_nesting_depth             int64
num_try_blocks                int64
has_large_return               bool
estimated_complexity          int64
dtype: object

In [29]:
y_train.dtypes

quality    category
dtype: object

# Defining Helper Functions

In [30]:
def plot_learning_curve(model, X, y, cv=5, title="Learning Curve"):
	train_sizes, train_scores, val_scores = learning_curve(
		model, X, y, cv=cv, scoring='accuracy',
		train_sizes=np.linspace(0.1, 1.0, 10),
		n_jobs=-1
	)
	
	plt.figure(figsize=(10, 6))
	plt.plot(
		train_sizes, np.mean(train_scores, axis=1), label='Training Score'
	)
	plt.plot(
		train_sizes, np.mean(val_scores, axis=1), label='Validation Score'
	)
	plt.fill_between(
		train_sizes, np.mean(train_scores, axis=1) - np.std(train_scores, axis=1),
		np.mean(train_scores, axis=1) + np.std(train_scores, axis=1), alpha=0.1
	)
	plt.fill_between(
		train_sizes, np.mean(val_scores, axis=1) - np.std(val_scores, axis=1),
		np.mean(val_scores, axis=1) + np.std(val_scores, axis=1), alpha=0.1
	)
	
	plt.title(title)
	plt.xlabel("Training Examples")
	plt.ylabel("Accuracy Score")
	plt.legend()
	plt.grid()
	plt.show()

# Data Preprocessing

## Converting Boolean Columns to Numerical (0, 1)

In [31]:
bool_cols = X_train.select_dtypes(include='bool').columns
X_train[bool_cols] = X_train[bool_cols].astype(int)
X_test[bool_cols] = X_test[bool_cols].astype(int)

## Scaling the Data

### Identifying numerical columns (excluding boolean/binary columns)

In [32]:
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
numerical_cols = [col for col in numerical_cols if col not in bool_cols]

### Scaling

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('bool', 'passthrough', bool_cols)  # Skip scaling for boolean columns
    ],
    remainder='passthrough'  # This will handle any other columns if they exist
)

X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

### Converting to DataFrame to maintain column names

In [34]:
feature_names = numerical_cols + list(bool_cols)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_names)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_names)

## Encoding the Target Variable

In [35]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train.squeeze())

# Model Training

In [36]:
pipeline = Pipeline(steps=[
    ('classifier', LogisticRegression(
        multi_class='auto',
        class_weight='balanced', # blances data to account for class imbalance if there are any
        solver='saga', # better than 'lbfgs' for out use case since our data is large
        max_iter=1000, # max imum number of iterations before convergence
        n_jobs=-1,            # parallelize computation
        random_state=42
    ))
])      

In [37]:
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)

c:\Users\Serio\OneDrive\Desktop\Projects\School\Machine\code-quality-assessor\code-quality-assessor\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Serio\OneDrive\Desktop\Projects\School\Machine\code-quality-assessor\code-quality-assessor\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\Serio\OneDrive\Desktop\Projects\School\Machine\code-quality-assessor\code-quality-assessor\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [38]:
print("Classification Report:")
print(classification_report(
    y_test,
    y_pred,
    target_names=["bad", "good"]
))

Classification Report:
              precision    recall  f1-score   support

         bad       0.69      0.74      0.72     69808
        good       0.73      0.68      0.71     72102

    accuracy                           0.71    141910
   macro avg       0.71      0.71      0.71    141910
weighted avg       0.71      0.71      0.71    141910

